<a href="https://colab.research.google.com/github/basmasaidi00/NPLtp/blob/main/Classification_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importation du dataset


In [ ]:
!pip install -q Kaggle

In [ ]:
!mkdir -p ~/.kaggle/

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/

In [ ]:
! kaggle datasets download -d nltkdata/movie-review

 57% 4.00M/7.07M [00:00<00:00, 6.31MB/s]
100% 7.07M/7.07M [00:01<00:00, 7.36MB/s]


In [13]:
!unzip /content/movie-review.zip

Archive:  /content/movie-review.zip
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

#Importer les bibliotèques

In [22]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


charger le dataset

In [18]:
df = pd.read_csv("movie_review.csv")

In [19]:
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


#Pre-processing des données textuelles

In [20]:
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les stopwords
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)

# Appliquer le prétraitement à la colonne Text
df['text'] = df['text'].apply(preprocess_text)

# Afficher les premières lignes du dataframe après prétraitement
print(df.head())

   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  
0  films adapted comic books plenty success wheth...  pos  
1  starters created alan moore eddie campbell bro...  pos  
2  say moore campbell thoroughly researched subje...  pos  
3  book graphic novel 500 pages long includes nea...  pos  
4                     words dont dismiss film source  pos  


#Entraînement du modèle Word2Vec

In [23]:
sentences = [nltk.word_tokenize(text) for text in df['text']]
word2vec_model = Word2Vec(sentences, min_count=1)

#Vectorisation des reviews de movies

In [25]:
import numpy as np
def vectorize_text(text, model):
    vectors = [model.wv[word] for word in text.split() if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df['Vectorized_Text'] = df['text'].apply(lambda x: vectorize_text(x, word2vec_model))

#Dévision du dataset

In [26]:
from sklearn.model_selection import train_test_split
X = np.stack(df['Vectorized_Text'])
y = df['tag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
X_train

array([[-0.33047482,  0.20135798,  0.17991816, ..., -0.50463825,
         0.1304958 ,  0.08066712],
       [-0.17999449,  0.26911762,  0.22378032, ..., -0.3636837 ,
         0.27098614,  0.16888128],
       [-0.31959113,  0.28880927,  0.2354233 , ..., -0.38213733,
         0.13236614,  0.21749707],
       ...,
       [-0.17991275,  0.26975384,  0.21951218, ..., -0.42349452,
         0.32213414,  0.14763437],
       [-0.25429124,  0.25404063,  0.16090475, ..., -0.44114366,
         0.18960974,  0.06774186],
       [-0.18210363,  0.2198256 ,  0.16733067, ..., -0.3283354 ,
         0.12229236,  0.13784388]])

#Construction d'un classificateur :

In [28]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

#Évaluation du modèle

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Prédiction sur l'ensemble de test
y_pred = classifier.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


Accuracy: 0.5694530284301607
Precision: 0.5706714421344781
Recall: 0.5694530284301607
F1 score: 0.5650166522527154
